In [ ]:
from enum import Enum
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2 as cv
import numpy as np
from cleanvision import Imagelab
from cv2.typing import MatLike

df = pd.read_csv("Dataset/meta_train.csv")

print("Initial shape of the dataset: ", df.shape)

imagelab = Imagelab(data_path="Dataset/train_images")
imagelab.find_issues()

imagelab.info["statistics"]["aspect_ratio"]

imagelab.info["statistics"]["size"]

imagelab.issue_summary

exact_duplicate = imagelab.issues[
    imagelab.issues["is_exact_duplicates_issue"]
].index.tolist()
near_duplicate = imagelab.issues[
    imagelab.issues["is_near_duplicates_issue"]
].index.tolist()
imagelab.visualize(image_files=exact_duplicate[:8])
imagelab.visualize(image_files=near_duplicate[:8])

unreliable_sets = []

for set in imagelab.info["exact_duplicates"]["sets"]:
    image_1, image_2 = [set[0].split("/")[-1], set[1].split("/")[-1]]
    meta_1 = (
        df[df["image_id"] == image_1].reset_index().drop(["image_id", "index"], axis=1)
    )
    meta_2 = (
        df[df["image_id"] == image_2].reset_index().drop(["image_id", "index"], axis=1)
    )
    # if comparision has value
    if not meta_1.compare(meta_2).empty:
        unreliable_sets.append(set)
        print("different metadata")

for set in imagelab.info["near_duplicates"]["sets"]:
    image_1, image_2 = [set[0].split("/")[-1], set[1].split("/")[-1]]
    meta_1 = (
        df[df["image_id"] == image_1].reset_index().drop(["image_id", "index"], axis=1)
    )
    meta_2 = (
        df[df["image_id"] == image_2].reset_index().drop(["image_id", "index"], axis=1)
    )
    # if comparision has value
    if not meta_1.compare(meta_2).empty:
        unreliable_sets.append(set)
        print("different metadata")

for set in imagelab.info["exact_duplicates"]["sets"]:
    for i in range(len(set)):
        if i != 0:
            df = df[df["image_id"] != set[i].split("/")[-1]]

for set in imagelab.info["near_duplicates"]["sets"]:
    for i in range(len(set)):
        if i != 0:
            df = df[df["image_id"] != set[i].split("/")[-1]]

for set in unreliable_sets:
    for i in range(len(set)):
        df = df[df["image_id"] != set[i].split("/")[-1]]
        
print("Final shape of the dataset: ", df.shape)

In [ ]:
import tensorflow as tf
from tensorflow.python.keras import mixed_precision
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input, GlobalAveragePooling2D, Concatenate

def create_gpu_optimized_model(input_shape, num_classes):
    inputs = Input(shape=input_shape, dtype=tf.float16)
    
    # Large kernel initial layers to utilize GPU parallelism
    x = Conv2D(96, (11,11), strides=4, activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D((3,3), strides=2)(x)
    
    # Bottleneck layers for memory efficiency
    x = Conv2D(256, (5,5), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((3,3), strides=2)(x)
    
    # Parallel convolutions
    branch1 = Conv2D(384, (3,3), activation='relu', padding='same')(x)
    branch2 = Conv2D(384, (3,3), dilation_rate=2, activation='relu', padding='same')(x)
    x = Concatenate()([branch1, branch2])
    
    # Final layers
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax', dtype=tf.float32)(x)
    
    return tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
def calculate_max_batch_size(model, input_shape=(480, 320, 3), gpu_mem=24):
    """Calculate maximum batch size for 24GB VRAM"""
    from keras import backend as K
    import numpy as np
    
    # Memory usage per sample (empirical formula)
    params = model.count_params()
    activations = np.prod(input_shape) * model.layers[-2].units  # Last dense layer
    per_sample = (params * 2 + activations * 4) / (1024 ** 3)  # GB
    
    # Available VRAM (with 2GB buffer)
    usable_mem = gpu_mem - 2
    
    max_batch = int(usable_mem / per_sample)
    return max(16, min(256, max_batch))  # Keep within reasonable bounds

# Example usage:
model = create_gpu_optimized_model((480, 320, 3), num_classes=10)
recommended_batch = calculate_max_batch_size(model)
print(f"Recommended batch size: {recommended_batch}")

In [ ]:
import tensorflow as tf
from tensorflow.keras import mixed_precision
import numpy as np
import os
import cv2
import albumentations as A
from sklearn.preprocessing import LabelEncoder

# === Configuration ===
config = {
    "data_path": "Dataset/train_images",
    "csv_path": "Dataset/meta_train.csv",
    "target_size": (480, 320),  # 3:2 ratio
    "epochs": 100,
    "initial_lr": 0.001,
    "gpu_memory_limit": 24  # GB
}

# === GPU Setup ===
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Enable mixed precision
        policy = mixed_precision.Policy('mixed_float16')
        mixed_precision.set_global_policy(policy)
        
        # Memory growth and optimization
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.optimizer.set_jit(True)
        tf.config.threading.set_intra_op_parallelism_threads(8)
        tf.config.threading.set_inter_op_parallelism_threads(4)
    except RuntimeError as e:
        print(e)

# === Data Preparation ===
def load_and_preprocess_data():
    df = pd.read_csv(config["csv_path"])
    
    # Label encoding
    le = LabelEncoder()
    df['label_encoded'] = le.fit_transform(df['label'])
    
    # Train-val split
    from sklearn.model_selection import train_test_split
    train_df, val_df = train_test_split(df, test_size=0.15, stratify=df['label'])
    
    return train_df, val_df, len(le.classes_)

# === Data Generator ===
class RiceDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, base_path, batch_size=32, shuffle=True):
        self.df = df
        self.base_path = base_path
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(df))
        self.aug = A.Compose([
            A.RandomRotate90(),
            A.Flip(),
            A.Transpose(),
            A.RandomBrightnessContrast(p=0.5),
            A.HueSaturationValue(p=0.5),
            A.CLAHE(p=0.5),
        ])
        if shuffle:
            np.random.shuffle(self.indices)
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, idx):
        batch_indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_df = self.df.iloc[batch_indices]
        
        X = np.zeros((len(batch_df), *config["target_size"], 3), dtype=np.float16)
        y = np.zeros((len(batch_df),), dtype=np.int32)
        
        for i, (_, row) in enumerate(batch_df.iterrows()):
            img = cv2.imread(os.path.join(self.base_path, row['image_id']))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = self.aug(image=img)['image']
            X[i] = img.astype(np.float16) / 255.0
            y[i] = row['label_encoded']
            
        return X, y

# === Model Training ===
def train():
    # Load data
    train_df, val_df, num_classes = load_and_preprocess_data()
    
    # Create model
    model = create_gpu_optimized_model((*config["target_size"], 3), num_classes)
    
    # Calculate optimal batch size
    batch_size = calculate_max_batch_size(model)
    print(f"\n=== Training Configuration ===")
    print(f"Batch size: {batch_size}")
    print(f"Input size: {config['target_size']}")
    print(f"GPU Memory: {config['gpu_memory_limit']}GB\n")
    
    # Create generators
    train_gen = RiceDataGenerator(train_df, config["data_path"], batch_size=batch_size)
    val_gen = RiceDataGenerator(val_df, config["data_path"], batch_size=batch_size, shuffle=False)
    
    # Compile
    model.compile(
        optimizer=tf.keras.optimizers.AdamW(learning_rate=config["initial_lr"], weight_decay=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy',
                 tf.keras.metrics.AUC(name='auc'),
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall')]
    )
    
    # Callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3),
        tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True),
        tf.keras.callbacks.TensorBoard(log_dir='./logs')
    ]
    
    # Train
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=config["epochs"],
        callbacks=callbacks,
        verbose=1
    )
    
    return model, history

# === Execution ===
if __name__ == "__main__":
    model, history = train()